In [43]:
import tensorflow as tf
from tensorflow import keras
import IPython

# !pip install -q -U keras-tuner
import keras_tuner as kt

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [44]:
seed_num = 48

import random    
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4069), (6182,), (1545, 10, 4069), (1545,))

In [45]:
def model_builder(hp):
    model = Sequential()
    model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    
    hp_units1 = hp.Int('units1', min_value = 32, max_value = 512, step = 32)
    hp_units2 = hp.Int('units2', min_value = 32, max_value = 512, step = 32)
    hp_units3 = hp.Int('units3', min_value = 32, max_value = 512, step = 32)
    hp_units4 = hp.Int('units4', min_value = 32, max_value = 512, step = 32)
    
    hp_activ1 = hp.Choice('activation1', values=['relu', 'tanh', 'hard_sigmoid'], default='hard_sigmoid')
    hp_activ2 = hp.Choice('activation2', values=['relu', 'tanh', 'hard_sigmoid'], default='hard_sigmoid')
    hp_activ3 = hp.Choice('activation3', values=['relu', 'tanh', 'hard_sigmoid'], default='hard_sigmoid')
    hp_activ4 = hp.Choice('activation4', values=['relu', 'tanh', 'hard_sigmoid'], default='hard_sigmoid')
    
    hp_drop1 = hp.Float('dropout1', min_value=0.0,max_value=0.5,default=0.2,step=0.05)
    hp_drop2 = hp.Float('dropout2', min_value=0.0,max_value=0.5,default=0.2,step=0.05)
    
    hp_activ_dense = hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid'], default='sigmoid')
    
    model.add(LSTM(units = hp_units1, activation = hp_activ1, return_sequences=True))
    model.add(LSTM(units = hp_units2, activation = hp_activ2, return_sequences=True))
    model.add(Dropout(rate=hp_drop1))
    model.add(LSTM(units = hp_units3, activation = hp_activ3, return_sequences=True))
    model.add(LSTM(units = hp_units4, activation = hp_activ4, return_sequences=False))
    model.add(Dropout(rate=hp_drop2))
    model.add(Dense(1, activation=hp_activ_dense))

    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = "binary_crossentropy", 
                metrics = ['accuracy'])
    
    return model

In [46]:
ep = 500
pa = 10
early_stop = EarlyStopping(monitor='val_acc', patience=pa, verbose=1, restore_best_weights=True)
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = ep,
                     factor = 3,
                     directory = 'my_dir1',
                     project_name = 'intro_to_kt')

In [47]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

In [ ]:
%%time
tuner.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

Trial 48 Complete [00h 01m 28s]
val_accuracy: 0.6284789443016052

Best val_accuracy So Far: 0.8019417524337769
Total elapsed time: 01h 02m 18s

Search: Running Trial #49

Hyperparameter    |Value             |Best Value So Far 
units1            |448               |128               
units2            |384               |64                
units3            |320               |352               
units4            |96                |320               
activation1       |tanh              |hard_sigmoid      
activation2       |hard_sigmoid      |relu              
activation3       |hard_sigmoid      |hard_sigmoid      
activation4       |hard_sigmoid      |hard_sigmoid      
dropout1          |0.2               |0.1               
dropout2          |0.05              |0.05              
dense_activation  |sigmoid           |sigmoid           
learning_rate     |0.001             |0.001             
tuner/epochs      |3                 |3                 
tuner/initial_e...|0           

In [21]:
# Build the model with the optimal hyperparameters and train it on the data
early_stop = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, restore_best_weights=True)

model = tuner.hypermodel.build(best_hps)
model.fit(X_train, y_train, epochs = 500, validation_data = (X_test, y_test),  callbacks=[early_stop], shuffle=False)

Epoch 1/500
194/194 [==============================] - 18s 68ms/step - loss: 0.6420 - accuracy: 0.6286 - val_loss: 0.5101 - val_accuracy: 0.7515
Epoch 2/500
194/194 [==============================] - 13s 66ms/step - loss: 0.4676 - accuracy: 0.7737 - val_loss: 0.5005 - val_accuracy: 0.7689
Epoch 3/500
194/194 [==============================] - 12s 63ms/step - loss: 0.3982 - accuracy: 0.8204 - val_loss: 0.5322 - val_accuracy: 0.7566
Epoch 4/500
194/194 [==============================] - 10s 51ms/step - loss: 0.3621 - accuracy: 0.8423 - val_loss: 0.5495 - val_accuracy: 0.7631
Epoch 5/500
194/194 [==============================] - 10s 53ms/step - loss: 0.3342 - accuracy: 0.8610 - val_loss: 0.4992 - val_accuracy: 0.7786
Epoch 6/500
194/194 [==============================] - 10s 54ms/step - loss: 0.2920 - accuracy: 0.8835 - val_loss: 0.5831 - val_accuracy: 0.7508
Epoch 7/500
194/194 [==============================] - 9s 47ms/step - loss: 0.2644 - accuracy: 0.8949 - val_loss: 0.5343 - val_acc

In [22]:
preds = model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))

정확도 : 0.7786407766990291
